# Text simplifcation model
**Credits**: This work has been adapted from the example code provided in the `Transformers` library released under the `Apache license`.

In [ ]:
# ! pip install -q datasets evaluate rouge-score nltk transformers==4.30 sentencepiece # set to previous version due to error which says evaluate not found

## Fine-tuning T5-base model

In [ ]:
import os

model_name = "t5-base"
model_dir = f"{model_name.replace('/', '-')}-super-checkpoints"

# Check if model directory exists and is not empty
if os.path.exists(model_dir) and any([item.startswith('checkpoint-') for item in os.listdir(model_dir)]):
    model_checkpoint = os.path.join(model_dir, os.listdir(model_dir)[-1])
else:
    model_checkpoint = model_name

## Loading the dataset

We will use the [Datasets](https://github.com/huggingface/datasets) library to process our data and use the [Evaluate](https://github.com/huggingface/evaluate) get the metric we need to use for evaluation.

In [ ]:
from datasets import load_dataset

data_dir = f'{parent_dir}/data'
raw_datasets = load_dataset("csv", data_files=f'{data_dir}/data.tsv', delimiter='\t')

all_columns = raw_datasets.column_names['train']
required_columns = ['original', 'english super simplified']
unrequired_columns = [col for col in all_columns if col not in required_columns]
original_col = required_columns[0]
target_col = required_columns[-1]

# Remove unrequired columns
raw_datasets['train'] = raw_datasets['train'].remove_columns(unrequired_columns)

# Filter out rows where 'english super simplified' is None
raw_datasets['train'] = raw_datasets['train'].filter(lambda example: example['english super simplified'] is not None)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets["train"][0]

## Dataset train/validation/test split

We split the dataset in the below ratio:
- Training set: 99%
- Validation set: 0.5%
- Test set: 0.5%

In [ ]:
# Calculate sizes for train, validation, and test sets
total_n = raw_datasets['train'].num_rows
split_n = int(0.005 * total_n)

# Define indices for train, validation, and test splits
train_indices = list(range(total_n - 2 * split_n))
validation_indices = list(range(total_n - 2 * split_n, total_n - split_n))
test_indices = list(range(total_n - split_n, total_n))

# Perform rigid train-validation-test split
raw_datasets["validation"] = raw_datasets["train"].select(indices=validation_indices).shuffle(seed=42)
raw_datasets["test"] = raw_datasets["train"].select(indices=test_indices).shuffle(seed=42)
raw_datasets["train"] = raw_datasets["train"].select(indices=train_indices).shuffle(seed=42)

# Display raw_datasets to verify the splits
print(raw_datasets)

# used later for tokenization
max_input_length = 512
max_target_length = 512

In [ ]:
# # keep only a subsample of the datasets
# raw_datasets["train"] = raw_datasets["train"].select(range(10))
# raw_datasets["validation"] = raw_datasets["validation"].select(range(1))
# raw_datasets["test"] = raw_datasets["test"].select(range(1))

# raw_datasets

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

To prepare the targets for our model, we need to tokenize them using the `text_target` parameter. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
def preprocess_function(examples):
    inputs = [f'summarization: {original}' for original in examples[original_col]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples[target_col], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
# tokenized_datasets = tokenized_datasets.filter(lambda example: len(example['labels']) < max_target_length)
tokenized_datasets_reduced = tokenized_datasets.filter(lambda example: len(example['labels']) > 5 and len(example['labels']) < 256)

In [ ]:
# Save tokenized_datasets to disk as it is time-consuming to tokenize
tokenized_datasets_reduced.save_to_disk(f'{data_dir}/tokenized_datasets_reduced_en_super')

**We observe that our reduced dataset with only 256 tokens are sufficient to cover most records as below.**

This was done to save memory and improve training speed.
* 62.7k / 63.7k (98.4%)

In [ ]:
import pandas as pd

df = pd.Series([len(label) for label in tokenized_datasets_reduced['train']['labels']])
df.describe()

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_datasets = load_from_disk(f'{data_dir}/tokenized_datasets_reduced_en_super') # Load tokenized_datasets from disk
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import numpy as np
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
batch_size = 32

training_args = Seq2SeqTrainingArguments(
    model_dir,
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    save_strategy="steps",
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,
    generation_max_length=max_target_length, 
    # push_to_hub=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

We can now finetune our model by just calling the `train` method:

## Model evaluation

We load the best checkpoint for the model and evaluate its performance against other similar medical text simplification models.

In [ ]:
import os
import nltk
from transformers import AutoModelForSeq2SeqLM

model_name = "t5-base"
model_dir = f"{model_name.replace('/', '-')}-super-checkpoints"

# Check if model directory exists and is not empty
if os.path.exists(model_dir) and os.listdir(model_dir):
    model_checkpoint = os.path.join(model_dir, os.listdir(model_dir)[-1])
else:
    model_checkpoint = model_name

model_t = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_datasets = load_from_disk(f'{data_dir}/tokenized_datasets_reduced_en_super') # Load tokenized_datasets from disk

test_subset = tokenized_datasets['test'].shuffle().select(range(10))
test_sources = [r[original_col] for r in test_subset]
test_references = [r[target_col] for r in test_subset]
print(f"Sample Input: {test_sources[0]}")

In [ ]:
import torch

model_device = "cuda" if torch.cuda.is_available() else "cpu"
model_t.to(model_device)


tokenized_inputs = tokenizer(test_sources, max_length=max_input_length, truncation=True, padding=True, return_tensors="pt").to(model_device)
output = model_t.generate(**tokenized_inputs, num_beams=8, do_sample=True, min_length=0, max_length=max_target_length)

prediction = tokenizer.batch_decode(output, skip_special_tokens=True)
# prediction = [nltk.sent_tokenize(o.strip())[0] for o in decoded_output]

In [ ]:
import pandas as pd
import numpy as np

predictions_df = pd.DataFrame(np.array([test_sources, test_references, prediction]).T, columns=['Original sentence', 'Simplified sentence', 'Predicted sentence'])
predictions_df.to_csv(f'{results_dir}/predictions-10-{model_name.replace("/", "-")}-super.tsv', sep='\t')

In [ ]:
predictions_df

In [ ]:
from metrics import fk, ari, bleu, rouge, meteor, sari, bertscore

# Calculate metrics
fk_score = fk(prediction)
ari_score = ari(prediction)
bleu_score = bleu(test_references, prediction)
rouge_score = rouge(test_references, prediction)
meteor_score = meteor(test_references, prediction)
sari_score = sari(test_sources, test_references, prediction)
bertscore_score = bertscore(test_references, prediction)

print("FK index:", fk_score)
print("ARI index:", ari_score)
print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print("METEOR Score:", meteor_score)
print("SARI Score:", sari_score)
print("BERTScore:", bertscore_score)

# NOTE: There can be minor variance in results every time the evaluation is run, a mean of many samples is recommeded.

|                          | Readability |       | Lexical  |        |         |       | Simplification | Semantic  |
|--------------------------|:-----------:|:-----:|:--------:|:------:|:-------:|:-----:|:--------------:|:---------:|
| Models                   | FK          | ARI   | Rouge1   | Rouge2 | Rouge-L | BLEU  | SARI           | BertScore |
| UL-BART (Devraj et al.)  | 11.97       | 13.73 | 38.00    | 14.00  | 36.00   | 39.0  | 40.00          | N/A       |
| NapSS (Lu et al.)        | 10.97       | 14.27 | 48.05    | 19.94  | 44.76   | 12.3  | 40.37          | 25.73     |
| T5-base-simplified (ours) | 11.09       | 13.76 | **58.66**    | **33.75**  | **45.47**   | **27.7**  | **52.13**          | **74.83**     |
| T5-base-extra-simplified (ours) | **10.01**       | **12.1** | 31.68    | 10.68  | 22.66   | 7.59  | 43.58          | 63.51     |

## The End
Thank you!!